In [53]:
# # This Python 3 environment comes with many helpful analytics libraries installed
# # It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# # For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# # You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# # You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [54]:
!pip install datasets
!pip install transformers
!pip install bitsandbytes
!pip install trl
!pip install --upgrade numpy
!pip install peft

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

In [55]:
!pip install blingfire

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [56]:
import numpy as np
import pandas as pd 
from datasets import load_dataset, load_from_disk
from sklearn.feature_extraction.text import TfidfVectorizer
import torch
from transformers import LongformerTokenizer, LongformerForMultipleChoice
import transformers
import pandas as pd
import pickle
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
import unicodedata
import os, time
import gc
import pandas as pd
import numpy as np
import re
from tqdm.auto import tqdm
import blingfire as bf
from __future__ import annotations

from collections.abc import Iterable

# import faiss
# from faiss import write_index, read_index

# from sentence_transformers import SentenceTransformer

import torch
import ctypes
libc = ctypes.CDLL("libc.so.6")

from dataclasses import dataclass
from typing import Optional, Union

import torch
import numpy as np
import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer
from transformers import AutoModelForMultipleChoice, TrainingArguments, Trainer
from transformers.tokenization_utils_base import PreTrainedTokenizerBase, PaddingStrategy
from torch.utils.data import DataLoader

from scipy.special import softmax

import os

In [57]:
df = pd.read_csv('/kaggle/input/sih-2023/sample1.csv')
df.head()

,inLanguage,headline,description,datePublished,articleBody,image__@type,image__url,image__height,image__width,department
0,en,Karnataka govt organises mega event of reading...,The Karnataka government organized a large-sca...,2023-09-15T11:58:00+05:30,BENGALURU: The Karnataka government on Friday ...,ImageObject,https://economictimes.indiatimes.com/thumb/msi...,1280.0,1900.0,indian_government
1,NaN,NaN,"In view of the Nipah virus cases in Kerala, th...",NaN,NaN,NaN,NaN,NaN,NaN,indian_government
2,en,ET Soonicorns Summit: Govt needs to update pol...,The minister cited the 28% GST on online gamin...,2023-08-27T15:52:00+05:30,"Priyank Kharge, Karnataka's minister for infor...",ImageObject,https://economictimes.indiatimes.com/thumb/msi...,4480.0,6720.0,indian_government
3,en,Govt losing revenue due to GST: Bibek Debroy,"The ideal GST is one that has a single rate, ...",2023-08-22T15:36:00+05:30,Economic Advisory Council to the Prime Ministe...,ImageObject,https://economictimes.indiatimes.com/thumb/msi...,1280.0,1767.0,indian_government
4,en,"'Personal view, not ours' says govt on EAC-PM ...",The Economic Advisory Council to the Prime Min...,2023-08-18T14:00:00+05:30,The Economic Advisory Council to the Prime Min...,ImageObject,https://economictimes.indiatimes.com/thumb/msi...,1280.0,1903.0,indian_government


In [58]:
(df[df['inLanguage'] != 'en'])
df['inLanguage'] = 'en'

In [59]:
(df[df['inLanguage'] != 'en'])

,inLanguage,headline,description,datePublished,articleBody,image__@type,image__url,image__height,image__width,department


In [60]:
df.dropna()

,inLanguage,headline,description,datePublished,articleBody,image__@type,image__url,image__height,image__width,department
0,en,Karnataka govt organises mega event of reading...,The Karnataka government organized a large-sca...,2023-09-15T11:58:00+05:30,BENGALURU: The Karnataka government on Friday ...,ImageObject,https://economictimes.indiatimes.com/thumb/msi...,1280.0,1900.0,indian_government
2,en,ET Soonicorns Summit: Govt needs to update pol...,The minister cited the 28% GST on online gamin...,2023-08-27T15:52:00+05:30,"Priyank Kharge, Karnataka's minister for infor...",ImageObject,https://economictimes.indiatimes.com/thumb/msi...,4480.0,6720.0,indian_government
3,en,Govt losing revenue due to GST: Bibek Debroy,"The ideal GST is one that has a single rate, ...",2023-08-22T15:36:00+05:30,Economic Advisory Council to the Prime Ministe...,ImageObject,https://economictimes.indiatimes.com/thumb/msi...,1280.0,1767.0,indian_government
4,en,"'Personal view, not ours' says govt on EAC-PM ...",The Economic Advisory Council to the Prime Min...,2023-08-18T14:00:00+05:30,The Economic Advisory Council to the Prime Min...,ImageObject,https://economictimes.indiatimes.com/thumb/msi...,1280.0,1903.0,indian_government
5,en,Punjab government advises people of five distr...,The Punjab government on Monday warned residen...,2023-08-14T21:38:00+05:30,The Punjab government on Monday warned residen...,ImageObject,https://economictimes.indiatimes.com/thumb/msi...,450.0,600.0,indian_government
...,...,...,...,...,...,...,...,...,...,...
227,en,Launch 'PLI Plus' initiative; focus on new pro...,The government should consider launching a 'PL...,2023-04-23T21:11:00+05:30,The government should consider launching a 'PL...,ImageObject,https://economictimes.indiatimes.com/thumb/msi...,223.0,300.0,department_of_science_and_technology
228,en,New space policy to boost private participatio...,Addressing the G20 Space Economy Leaders meeti...,2023-04-17T20:34:00+05:30,India has built end-to-end capability in the s...,ImageObject,https://economictimes.indiatimes.com/thumb/msi...,900.0,1200.0,department_of_science_and_technology
229,en,9 per cent of India's electricity to come from...,India plans to generate 9% of its electricity ...,2023-04-09T20:52:00+05:30,India will get 9 per cent of its electricity f...,ImageObject,https://economictimes.indiatimes.com/thumb/msi...,900.0,1200.0,department_of_science_and_technology
230,en,UK PM Rishi Sunak unveils new framework to mak...,UK PM Rishi Sunak said the new Science and Tec...,2023-03-06T17:25:00+05:30,British Prime Minister Rishi Sunak on Monday l...,ImageObject,https://economictimes.indiatimes.com/thumb/msi...,900.0,1200.0,department_of_science_and_technology


In [61]:
df = df.drop(columns = ['image__@type', 'image__height', 'image__width', 'image__url',])

In [62]:
df.head()

,inLanguage,headline,description,datePublished,articleBody,department
0,en,Karnataka govt organises mega event of reading...,The Karnataka government organized a large-sca...,2023-09-15T11:58:00+05:30,BENGALURU: The Karnataka government on Friday ...,indian_government
1,en,NaN,"In view of the Nipah virus cases in Kerala, th...",NaN,NaN,indian_government
2,en,ET Soonicorns Summit: Govt needs to update pol...,The minister cited the 28% GST on online gamin...,2023-08-27T15:52:00+05:30,"Priyank Kharge, Karnataka's minister for infor...",indian_government
3,en,Govt losing revenue due to GST: Bibek Debroy,"The ideal GST is one that has a single rate, ...",2023-08-22T15:36:00+05:30,Economic Advisory Council to the Prime Ministe...,indian_government
4,en,"'Personal view, not ours' says govt on EAC-PM ...",The Economic Advisory Council to the Prime Min...,2023-08-18T14:00:00+05:30,The Economic Advisory Council to the Prime Min...,indian_government


In [63]:
df['context'] = df['headline']+ '.' + df['description']+ '.' + df['datePublished'] + '.' + df['articleBody']

In [64]:
df.head()

,inLanguage,headline,description,datePublished,articleBody,department,context
0,en,Karnataka govt organises mega event of reading...,The Karnataka government organized a large-sca...,2023-09-15T11:58:00+05:30,BENGALURU: The Karnataka government on Friday ...,indian_government,Karnataka govt organises mega event of reading...
1,en,NaN,"In view of the Nipah virus cases in Kerala, th...",NaN,NaN,indian_government,NaN
2,en,ET Soonicorns Summit: Govt needs to update pol...,The minister cited the 28% GST on online gamin...,2023-08-27T15:52:00+05:30,"Priyank Kharge, Karnataka's minister for infor...",indian_government,ET Soonicorns Summit: Govt needs to update pol...
3,en,Govt losing revenue due to GST: Bibek Debroy,"The ideal GST is one that has a single rate, ...",2023-08-22T15:36:00+05:30,Economic Advisory Council to the Prime Ministe...,indian_government,Govt losing revenue due to GST: Bibek Debroy. ...
4,en,"'Personal view, not ours' says govt on EAC-PM ...",The Economic Advisory Council to the Prime Min...,2023-08-18T14:00:00+05:30,The Economic Advisory Council to the Prime Min...,indian_government,"'Personal view, not ours' says govt on EAC-PM ..."


In [65]:
df = df.drop(columns = ['inLanguage', 'headline', 'description', 'datePublished', 'articleBody'])

In [66]:
df.head()

,department,context
0,indian_government,Karnataka govt organises mega event of reading...
1,indian_government,NaN
2,indian_government,ET Soonicorns Summit: Govt needs to update pol...
3,indian_government,Govt losing revenue due to GST: Bibek Debroy. ...
4,indian_government,"'Personal view, not ours' says govt on EAC-PM ..."


In [67]:
df.dropna()

,department,context
0,indian_government,Karnataka govt organises mega event of reading...
2,indian_government,ET Soonicorns Summit: Govt needs to update pol...
3,indian_government,Govt losing revenue due to GST: Bibek Debroy. ...
4,indian_government,"'Personal view, not ours' says govt on EAC-PM ..."
5,indian_government,Punjab government advises people of five distr...
...,...,...
227,department_of_science_and_technology,Launch 'PLI Plus' initiative; focus on new pro...
228,department_of_science_and_technology,New space policy to boost private participatio...
229,department_of_science_and_technology,9 per cent of India's electricity to come from...
230,department_of_science_and_technology,UK PM Rishi Sunak unveils new framework to mak...


In [68]:
df.head()

,department,context
0,indian_government,Karnataka govt organises mega event of reading...
1,indian_government,NaN
2,indian_government,ET Soonicorns Summit: Govt needs to update pol...
3,indian_government,Govt losing revenue due to GST: Bibek Debroy. ...
4,indian_government,"'Personal view, not ours' says govt on EAC-PM ..."


In [69]:
type(df['context'][1])

float

In [70]:
df = df.drop(df[df['context'].isna()].index)

In [71]:
df.head()

,department,context
0,indian_government,Karnataka govt organises mega event of reading...
2,indian_government,ET Soonicorns Summit: Govt needs to update pol...
3,indian_government,Govt losing revenue due to GST: Bibek Debroy. ...
4,indian_government,"'Personal view, not ours' says govt on EAC-PM ..."
5,indian_government,Punjab government advises people of five distr...


In [72]:
df = df.drop(df[df['department'] == 'indian_government'].index)

In [73]:
df.head()

,department,context
29,department_of_personnel_and_training,Police department wants grading of stations ba...
30,department_of_personnel_and_training,NYC's homeless pavement dwellers hidden away &...
31,department_of_personnel_and_training,Income Tax raids on Samajwadi Party leader Aza...
32,department_of_personnel_and_training,Oppn bloc INDIA slams BJP for 'celebrations' a...
33,department_of_personnel_and_training,Sikkim govt will bring back old pension scheme...


In [74]:
df

,department,context
29,department_of_personnel_and_training,Police department wants grading of stations ba...
30,department_of_personnel_and_training,NYC's homeless pavement dwellers hidden away &...
31,department_of_personnel_and_training,Income Tax raids on Samajwadi Party leader Aza...
32,department_of_personnel_and_training,Oppn bloc INDIA slams BJP for 'celebrations' a...
33,department_of_personnel_and_training,Sikkim govt will bring back old pension scheme...
...,...,...
227,department_of_science_and_technology,Launch 'PLI Plus' initiative; focus on new pro...
228,department_of_science_and_technology,New space policy to boost private participatio...
229,department_of_science_and_technology,9 per cent of India's electricity to come from...
230,department_of_science_and_technology,UK PM Rishi Sunak unveils new framework to mak...


In [75]:
df = df.reset_index(drop=True)
df.head()

,department,context
0,department_of_personnel_and_training,Police department wants grading of stations ba...
1,department_of_personnel_and_training,NYC's homeless pavement dwellers hidden away &...
2,department_of_personnel_and_training,Income Tax raids on Samajwadi Party leader Aza...
3,department_of_personnel_and_training,Oppn bloc INDIA slams BJP for 'celebrations' a...
4,department_of_personnel_and_training,Sikkim govt will bring back old pension scheme...


In [76]:
len(df)

201

In [77]:
departments = [
    "Department for Promotion of Industry and Internal Trade",
    "Department of Administrative Reforms and Public Grievances (DARPG)",
    "Department of Agricultural Research and Education (DARE)",
    "Department of Agriculture and Farmers Welfare",
    "Department of Animal Husbandry and Dairying",
    "Department of Biotechnology",
    "Department of Border Management",
    "Department of Chemicals and Petrochemicals",
    "Department of Commerce",
    "Department of Consumer Affairs",
    "Department of Defence",
    "Department of Defence Production",
    "Department of Defence Research & Development",
    "Department of Drinking Water and Sanitation",
    "Department of Economic Affairs",
    "Department of Empowerment of Persons with Disabilities",
    "Department of Ex-Servicemen Welfare",
    "Department of Expenditure",
    "Department of Fertilizers",
    "Department of Financial Services",
    "Department of Fisheries",
    "Department of Food and Public Distribution",
    "Department of Health Research",
    "Department of Health and Family Welfare",
    "Department of Higher Education",
    "Department of Home",
    "Department of Investment and Public Asset Management",
    "Department of Justice",
    "Department of Land Resources (DLR)",
    "Department of Legal Affairs",
    "Department of Military Affairs (DMA)",
    "Department of Official Language",
    "Department of Pension & Pensioner's Welfare",
    "Department of Personnel and Training",
    "Department of Pharmaceuticals",
    "Department of Posts",
    "Department of Public Enterprises",
    "Department of Revenue",
    "Department of Rural Development (DRD)",
    "Department of School Education and Literacy",
    "Department of Science and Technology",
    "Department of Scientific and Industrial Research",
    "Department of Social Justice and Empowerment",
    "Department of Sports",
    "Department of Telecommunications",
    "Department of Water Resources, River Development and Ganga Rejuvenation",
    "Department of Youth Affairs",
    "Inter-State Council Secretariat",
    "Legislative Department"
]

# Create a list of dictionaries where each department is a choice
# multiple_choice_options = [{'label': department, 'value': department} for department in departments]

# Now, 'multiple_choice_options' contains the departments in the desired format


In [78]:
columns = [i for i in range(len(departments))]
# options = pd.Dataframe(departments)

In [79]:
# reshaped_columns = np.reshape(columns, (1, len(departments)))

In [80]:
len(columns) == len(departments)

True

In [81]:
matrix = [columns]*len(df)
# len(matrix)
# matrix

In [82]:
# matrix = np.reshape(matrix, (1,49,201))

In [83]:
# matrix.shape
# matrix[0][0]

In [84]:
data = pd.DataFrame(matrix, columns = departments)
data.head()

,Department for Promotion of Industry and Internal Trade,Department of Administrative Reforms and Public Grievances (DARPG),Department of Agricultural Research and Education (DARE),Department of Agriculture and Farmers Welfare,Department of Animal Husbandry and Dairying,Department of Biotechnology,Department of Border Management,Department of Chemicals and Petrochemicals,Department of Commerce,Department of Consumer Affairs,...,Department of School Education and Literacy,Department of Science and Technology,Department of Scientific and Industrial Research,Department of Social Justice and Empowerment,Department of Sports,Department of Telecommunications,"Department of Water Resources, River Development and Ganga Rejuvenation",Department of Youth Affairs,Inter-State Council Secretariat,Legislative Department
0,0,1,2,3,4,5,6,7,8,9,...,39,40,41,42,43,44,45,46,47,48
1,0,1,2,3,4,5,6,7,8,9,...,39,40,41,42,43,44,45,46,47,48
2,0,1,2,3,4,5,6,7,8,9,...,39,40,41,42,43,44,45,46,47,48
3,0,1,2,3,4,5,6,7,8,9,...,39,40,41,42,43,44,45,46,47,48
4,0,1,2,3,4,5,6,7,8,9,...,39,40,41,42,43,44,45,46,47,48


In [85]:
columns = df.iloc[50].values
columns[0]

'department_of_legal_affair'

In [86]:
# df[df['departments'] == ]
myset = set()
for i in df["department"]:
    myset.add(i)
    
myset

{'department_for_promotion_of_industry',
 'department_of_food',
 'department_of_health',
 'department_of_legal_affair',
 'department_of_personnel_and_training',
 'department_of_science_and_technology',
 'department_of_sports'}

In [87]:
df.head()

,department,context
0,department_of_personnel_and_training,Police department wants grading of stations ba...
1,department_of_personnel_and_training,NYC's homeless pavement dwellers hidden away &...
2,department_of_personnel_and_training,Income Tax raids on Samajwadi Party leader Aza...
3,department_of_personnel_and_training,Oppn bloc INDIA slams BJP for 'celebrations' a...
4,department_of_personnel_and_training,Sikkim govt will bring back old pension scheme...


In [88]:
test_df = df
test_df.loc[test_df['department'] == "department_for_promotion_of_industry",'department'] = "Department for Promotion of Industry and Internal Trade"
test_df.loc[test_df['department'] == "department_of_personnel_and_training",'department'] = "Department of Personnel and Training"
test_df.loc[test_df['department'] == "department_of_food",'department'] = "Department of Food and Public Distribution"
test_df.loc[test_df['department'] == "department_of_legal_affair",'department'] = "Department of Legal Affairs "
test_df.loc[test_df['department'] == "department_of_health",'department'] = "Department of Health and Family Welfare"
test_df.loc[test_df['department'] == "department_of_science_and_technology",'department'] = "Department of Science and Technology "
test_df.loc[test_df['department'] == "department_of_sports",'department'] = "Department of Sports"

test_df.head()

,department,context
0,Department of Personnel and Training,Police department wants grading of stations ba...
1,Department of Personnel and Training,NYC's homeless pavement dwellers hidden away &...
2,Department of Personnel and Training,Income Tax raids on Samajwadi Party leader Aza...
3,Department of Personnel and Training,Oppn bloc INDIA slams BJP for 'celebrations' a...
4,Department of Personnel and Training,Sikkim govt will bring back old pension scheme...


In [89]:
# # for i in myset:

#     df[df['departments'] == "department_for_promotion_of_industry"] = "Department for Promotion of Industry and Internal Trade"

In [90]:
contexts = []
index_to_option = {i:departments[i] for i in range(len(departments))}
# for index in tqdm(range(df.shape[0])):
index_to_option

{0: 'Department for Promotion of Industry and Internal Trade',
 1: 'Department of Administrative Reforms and Public Grievances (DARPG)',
 2: 'Department of Agricultural Research and Education (DARE)',
 3: 'Department of Agriculture and Farmers Welfare',
 4: 'Department of Animal Husbandry and Dairying',
 5: 'Department of Biotechnology',
 6: 'Department of Border Management',
 7: 'Department of Chemicals and Petrochemicals',
 8: 'Department of Commerce',
 9: 'Department of Consumer Affairs',
 10: 'Department of Defence',
 11: 'Department of Defence Production',
 12: 'Department of Defence Research & Development',
 13: 'Department of Drinking Water and Sanitation',
 14: 'Department of Economic Affairs',
 15: 'Department of Empowerment of Persons with Disabilities',
 16: 'Department of Ex-Servicemen Welfare',
 17: 'Department of Expenditure',
 18: 'Department of Fertilizers',
 19: 'Department of Financial Services',
 20: 'Department of Fisheries',
 21: 'Department of Food and Public Dist

In [91]:
option_to_index = {departments[i]:i for i in range(len(departments))}
option_to_index

{'Department for Promotion of Industry and Internal Trade': 0,
 'Department of Administrative Reforms and Public Grievances (DARPG)': 1,
 'Department of Agricultural Research and Education (DARE)': 2,
 'Department of Agriculture and Farmers Welfare': 3,
 'Department of Animal Husbandry and Dairying': 4,
 'Department of Biotechnology': 5,
 'Department of Border Management': 6,
 'Department of Chemicals and Petrochemicals': 7,
 'Department of Commerce': 8,
 'Department of Consumer Affairs': 9,
 'Department of Defence': 10,
 'Department of Defence Production': 11,
 'Department of Defence Research & Development': 12,
 'Department of Drinking Water and Sanitation': 13,
 'Department of Economic Affairs': 14,
 'Department of Empowerment of Persons with Disabilities': 15,
 'Department of Ex-Servicemen Welfare': 16,
 'Department of Expenditure': 17,
 'Department of Fertilizers': 18,
 'Department of Financial Services': 19,
 'Department of Fisheries': 20,
 'Department of Food and Public Distribu

In [92]:
# df = test_df
test_df['department'] = test_df['department'].map(option_to_index)
test_df.head()

,department,context
0,33.0,Police department wants grading of stations ba...
1,33.0,NYC's homeless pavement dwellers hidden away &...
2,33.0,Income Tax raids on Samajwadi Party leader Aza...
3,33.0,Oppn bloc INDIA slams BJP for 'celebrations' a...
4,33.0,Sikkim govt will bring back old pension scheme...


In [93]:
df = test_df

In [94]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer
tokenizer = AutoTokenizer.from_pretrained('/kaggle/input/deberta-v3-large-hf-weights')
model = AutoModelForSequenceClassification.from_pretrained(
    "/kaggle/input/deberta-v3-large-hf-weights", num_labels=len(departments), id2label=index_to_option, label2id=option_to_index)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/opt/conda/lib/python3.10/site-packages/transformers/convert_slow_tokenizer.py:473: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at /kaggle/input/deberta-v3-large-hf-weights and are newly initialized: ['classifier.weight', 'pooler.dense.weight', 'pooler.dense.bias', 'classifier.bias']
You should 

In [95]:
model

DebertaV2ForSequenceClassification(
  (deberta): DebertaV2Model(
    (embeddings): DebertaV2Embeddings(
      (word_embeddings): Embedding(128100, 1024, padding_idx=0)
      (LayerNorm): LayerNorm((1024,), eps=1e-07, elementwise_affine=True)
      (dropout): StableDropout()
    )
    (encoder): DebertaV2Encoder(
      (layer): ModuleList(
        (0-23): 24 x DebertaV2Layer(
          (attention): DebertaV2Attention(
            (self): DisentangledSelfAttention(
              (query_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (key_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (value_proj): Linear(in_features=1024, out_features=1024, bias=True)
              (pos_dropout): StableDropout()
              (dropout): StableDropout()
            )
            (output): DebertaV2SelfOutput(
              (dense): Linear(in_features=1024, out_features=1024, bias=True)
              (LayerNorm): LayerNorm((1024,), eps=1e-07, element

In [96]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

def pre_process(example):
    first_sentence = "[CLS]" + example["context"] + "[SEP]"
    tokenized_example = tokenizer(first_sentence, max_length=256, truncation=True, padding=True)
    
#     # Check if 'department' is None and assign a default value if needed
#     if example['department'] is None:
#         tokenized_example['label'] = torch.tensor(0.0)  # Assign a default value (0.0 in this case)
#     else:
    tokenized_example['label'] = example['department']
    
    return tokenized_example


In [97]:
import numpy as np


def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [98]:
df.head()

,department,context
0,33.0,Police department wants grading of stations ba...
1,33.0,NYC's homeless pavement dwellers hidden away &...
2,33.0,Income Tax raids on Samajwadi Party leader Aza...
3,33.0,Oppn bloc INDIA slams BJP for 'celebrations' a...
4,33.0,Sikkim govt will bring back old pension scheme...


In [99]:
df[df['department'] == np.nan]

,department,context


In [100]:
# df['de']
from sklearn.model_selection import train_test_split

# Split the DataFrame into training (80%) and validation (20%) sets
train_df, val_df = train_test_split(df, test_size=0.2, random_state=42)

# 'train_df' contains 80% of the data for training
# 'val_df' contains 20% of the data for validation


In [101]:
import pandas as pd

# Assuming df is your original DataFrame
# Filter out rows where 'department' is not None
filtered_train_df = train_df[train_df['department'].notna()]
filtered_val_df = val_df[val_df['department'].notna()]
# Create the tokenized dataset from the filtered DataFrame
tokenized_test_dataset = Dataset.from_pandas(filtered_train_df[['department', 'context']]).map(pre_process, remove_columns=['department', 'context'])
tokenized_val_dataset = Dataset.from_pandas(filtered_val_df[['department', 'context']]).map(pre_process, remove_columns=['department', 'context'])


  0%|          | 0/111 [00:00<?, ?ex/s]

  0%|          | 0/33 [00:00<?, ?ex/s]

In [102]:
# # tokenized_dataset = df.apply(pre_process, axis=1, batched=True)
# tokenised_test_dataset = Dataset.from_pandas(train_df[['department', 'context']]).map(pre_process, remove_columns = ['department', 'context'])
# tokenised_val_dataset = Dataset.from_pandas(val_df[['department', 'context']]).map(pre_process, remove_columns = ['department', 'context'])

In [105]:
tokenized_test_dataset

Dataset({
    features: ['__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask', 'label'],
    num_rows: 111
})

In [109]:
tokenized_test_dataset = tokenized_test_dataset.remove_columns(["__index_level_0__"])
tokenized_val_dataset = tokenized_val_dataset.remove_columns(["__index_level_0__"])

In [111]:
# from datasets import Dataset

# # Create a new list to store modified examples
# modified_examples = []
count = 0
# # Iterate over the original dataset
for example in tokenized_test_dataset:
    # Check if 'label' is None and set it to 0 if it is
    if type(example['label']) is None:
        count+=1
    elif example['label'] is None:
        count+=1
    # Append the modified example to the new list
#     modified_examples.append(example)
count
# # Create a new dataset with the modified examples
# modified_test_dataset = Dataset.from_dict({'input_ids': [ex['input_ids'] for ex in modified_examples],
#                                           'token_type_ids': [ex['token_type_ids'] for ex in modified_examples],
#                                           'attention_mask': [ex['attention_mask'] for ex in modified_examples],
#                                           'label': [ex['label'] for ex in modified_examples]})

# # Now you can use `modified_test_dataset` for further processing


0

In [114]:
# tokenised_dataset = tokenised_dataset.remove_columns(["__index_level_0__"])
test_dataloader = DataLoader(tokenized_test_dataset, batch_size=1, shuffle=False, collate_fn=data_collator)
val_dataloader = DataLoader(tokenized_val_dataset, batch_size = 1, shuffle = False, collate_fn = data_collator)

In [115]:
tokenized_val_dataset

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'label'],
    num_rows: 33
})

In [121]:
!pip install evaluate

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 2.0 MB/s eta 0:00:00a 0:00:01


In [122]:
import evaluate

accuracy = evaluate.load("accuracy")

In [ ]:
# training_args = TrainingArguments(
#     output_dir="my_awesome_model",
#     learning_rate=2e-5,
#     per_device_train_batch_size=1,
#     per_device_eval_batch_size=2,
#     num_train_epochs=18,
#     weight_decay=0.01,
#     evaluation_strategy="steps",
#     save_strategy="steps",
#     report_to = 'none',
# #     loss = 'cross_entropy',
#     load_best_model_at_end=True,
# #     push_to_hub=True,
# )
training_args = TrainingArguments(
    warmup_ratio=0.1, 
    learning_rate=2e-5,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=2,
    num_train_epochs=20,
    report_to='none',
    output_dir = f'./checkpoints_{1}',
    overwrite_output_dir=True,
    fp16=True,
    gradient_accumulation_steps=8,
    logging_steps=25,
    evaluation_strategy='steps',
    eval_steps=25,
    save_strategy="steps",
    save_steps=25,
    load_best_model_at_end=False,
    metric_for_best_model='accuracy',
    lr_scheduler_type='cosine',
    weight_decay=0.01,
    save_total_limit=2,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_test_dataset,
    eval_dataset=tokenized_val_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
#     compute_loss = compute_loss,
    compute_metrics=compute_metrics,
)

trainer.train()


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss,Validation Loss,Accuracy
25,0.427200,0.638581,0.818182


/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
